# Summarize the results of disorder prediction

In [1]:
import numpy as np
import pandas as pd
from itertools import chain
from collections import Counter
from tqdm.notebook import tqdm
from pyscripts.config import path2
from pyscripts.datasets import Metadata
metadata = Metadata()

In [2]:
topidp = pd.Series(dict(
    W = -0.884, F = -0.697, Y = -0.510, I = -0.486, 
    M = -0.397, L = -0.326, V = -0.121, N = +0.007, 
    C = +0.020, T = +0.059, A = +0.060, G = +0.166, 
    R = +0.180, D = +0.192, H = +0.302, Q = +0.318,
    S = +0.341, K = +0.586, E = +0.736, P = +0.987,
))

In [3]:
def get_disorder_summary2(gcf):
    nc = pd.read_pickle(path2.data/'iupred2a'/'noncoding-intermediates'/f'{gcf}.pkl.bz2')
    cd = pd.read_pickle(path2.data/'iupred2a'/'cds-intermediates'/f'{gcf}.pkl.bz2')
    dat = {
        'acc': gcf,
        'summary': [
            {
                'len_thresh': len_thresh , #'prob_thresh': prob_thresh, 
                'counts': Counter(
                    (r, a)
                    for ent in filter(lambda e: len(e['loc']) >= len_thresh, chain(nc, cd))
                    for r, a in zip(ent['relfr'], ent['transl'])
                )
            }
            for len_thresh  in (30, 100, 300) 
            #for prob_thresh in (0, 0.5, 0.6, 0.7, 0.75, 0.8, 0.9)
        ]
    }
    return dat

In [4]:
from multiprocessing import Pool
topidp_summary = pd.DataFrame(
    columns=pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=['gcf','len_thresh']),
)

with Pool(100) as pool:    
    for dat in tqdm(pool.imap_unordered(get_disorder_summary2, metadata.acc['refseq']), total=len(metadata.acc)):
        gcf = dat['acc']
        for s in dat['summary']:
            len_thresh = s['len_thresh']
            aa_counts = pd.Series(s['counts']).unstack().T.reindex([*'ACDEFGHIKLMNPQRSTVWY']).fillna(0)
            topidp_summary[(gcf, len_thresh)] = (aa_counts / aa_counts.sum()).T @ topidp
            
pd.to_pickle(topidp_summary, path2.data/'topidp'/'summary.pkl.bz2')

  0%|          | 0/2624 [00:00<?, ?it/s]